In [1]:
import pandas as pd

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, Activation, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping
from keras.layers import Lambda

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

from tqdm import tqdm

# all the functions from helpers.py
from helpers_scenario2 import *

import pandas as pd

from joblib import Parallel, delayed, parallel_backend
import multiprocessing
from multiprocessing import Value
from tqdm.auto import tqdm

# all the functions from helpers.py
from helpers_scenario2 import *

from keras.layers import Dropout, LSTM
from keras.regularizers import l1, l2
from keras.layers import Activation
from sklearn.preprocessing import StandardScaler
from kerastuner.tuners import RandomSearch

from keras.layers import MaxPooling1D

import json
from keras.layers import Dropout, LSTM
from keras.layers import Activation
from sklearn.preprocessing import StandardScaler
from kerastuner.tuners import RandomSearch

from keras.layers import MaxPooling1D


C:\Users\xochipilli\AppData\Local\Temp\ipykernel_11236\1464098886.py:35: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [ ]:
scenario_folder = "../data/preprocessed/cleaned_and_prepro_improved/scenario_2/"
fold = 0

phys_folder, ann_folder = create_folder_structure(scenario_folder, fold)
annotations_folder = '../data/raw/scenario_2/fold_0/train/annotations/'
physiology_folder = "../data/preprocessed/cleaned_and_prepro_improved/scenario_2/fold_0/train/physiology/" 
zipped_files = zip_csv_files(annotations_folder, physiology_folder)

subjects, videos = get_subs_vids(physiology_folder)

num_cpu_cores = multiprocessing.cpu_count()

def process_files(annotation_file, physiology_file,):
    df_annotations = pd.read_csv(annotation_file)
    df_physiology = pd.read_csv(physiology_file)
    
    print(physiology_file)
    X, y, numeric_column_indices = preprocess(df_physiology, df_annotations, predictions_cols=['arousal','valence'], aggregate=None, window=[-10000, 10000], partition_window = 1)
    # print(X.shape, y.shape)
    
    save_files(X, y, annotation_file, phys_folder, ann_folder)
    
    return None

# zipped_files = zipped_files[115:]

## Process the files using the context manager
with parallel_backend('loky', n_jobs=num_cpu_cores // 3):
    with tqdm_joblib(tqdm(total=len(zipped_files), desc="Files", leave=False)) as progress_bar:
        results = Parallel()(
            (delayed(process_files)(ann_file, phys_file) for ann_file, phys_file in zipped_files)
        )

#for ann_file, phys_file in zipped_files:
#    process_files(ann_file, phys_file,)
    


In [2]:
scenario_folder = "../data/preprocessed/cleaned_and_prepro_improved/scenario_2/"
fold = 0

phys_folder, ann_folder = create_folder_structure(scenario_folder, fold)

In [3]:
print(phys_folder)

c:\Users\xochipilli\Documents\EPiC-2023-Challenge\data\preprocessed\cleaned_and_prepro_improved\scenario_2\fold_0\preprocessed\physiology


In [4]:
annotations_folder = '../data/raw/scenario_2/fold_0/train/annotations/'
physiology_folder = "../data/preprocessed/cleaned_and_prepro_improved/scenario_2/fold_0/train/physiology/" 
zipped_files = zip_csv_files(annotations_folder, physiology_folder)

subjects, videos = get_subs_vids(physiology_folder)

# Comment this lineas
#videos = [0]
#subjects = [0,1,11]

splits = split_subjects_train_test(subjects, 3)

In [5]:
print(subjects)

[26, 18, 31, 20, 13, 34, 32, 17, 2, 9, 15, 45, 0, 11, 7, 24, 5, 28, 41, 43, 27, 30, 1, 29]


In [6]:
def create_cnn_lstm_model(input_shape, lstm_units=64, dropout_rate=0.4, kernel_regularizer_l1=0.0032):
    input_signal = Input(shape=input_shape)

    x = Conv1D(16, 5, padding='same')(input_signal)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(dropout_rate)(x)

    x = MaxPooling1D(pool_size=2)(x)

    x = LSTM(lstm_units)(x)

    x = Dense(32, activation='relu', kernel_regularizer=l1(kernel_regularizer_l1))(x)

    return Model(inputs=input_signal, outputs=x)

In [ ]:
import os
import json

# Initialize an empty dictionary to store the fold results
fold_results = {}

# Create a directory to save the models
os.makedirs("saved_models", exist_ok=True)

# Loop through the splits
for split_index, split in enumerate(splits):
    
    X_train, X_test = load_and_concatenate_files(phys_folder, split, videos)
    y_train, y_test = load_and_concatenate_files(ann_folder, split, videos)


    print("X_test shape:", X_test.shape)
    print("y_test shape:", y_test.shape)
                
    # Extract arousal and valence values from y_train and y_test
    y_arousal_train = y_train[:, 0]
    y_valence_train = y_train[:, 1]
    y_arousal_test = y_test[:, 0]
    y_valence_test = y_test[:, 1]

    input_shape = (X_train.shape[1], X_train.shape[2])
    cnn_lstm_model = create_cnn_lstm_model(input_shape)

    # Create separate heads for valence and arousal prediction, using sigmoid activation and scaling the output
    valence_output = Dense(1, activation='sigmoid')(cnn_lstm_model.output)
    valence_output = Lambda(lambda x: x * 8 + 1, name='valence_output')(valence_output)

    arousal_output = Dense(1, activation='sigmoid')(cnn_lstm_model.output)
    arousal_output = Lambda(lambda x: x * 8 + 1, name='arousal_output')(arousal_output)

    # Combine the model
    final_model = Model(inputs=cnn_lstm_model.input, outputs=[valence_output, arousal_output])

    # Compile the model
    final_model.compile(optimizer='adam',
                        loss={'valence_output': 'mse',
                            'arousal_output': 'mse'},
                        metrics=[tf.keras.metrics.RootMeanSquaredError()])

    # Set up early stopping
    early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Train the model
    history = final_model.fit(X_train, {'valence_output': y_valence_train, 'arousal_output': y_arousal_train},
                              validation_split=0.2, epochs=50, batch_size=32,
                              callbacks=[early_stopping_callback])

    evaluation = final_model.evaluate(X_test, [y_valence_test, y_arousal_test])

    # Save the evaluation results in the dictionary
    key = f"split_{split_index}"
    fold_results[key] = {
        'valence_output_root_mean_squared_error': evaluation[3],
        'arousal_output_root_mean_squared_error': evaluation[4]
    }

    # Save the trained model for this split
    model_path = os.path.join("saved_models", f"model_split_{split_index}.h5")
    final_model.save(model_path)

# Calculate the average of the evaluation results
evaluation_sum = np.array([list(rmse_dict.values()) for rmse_dict in fold_results.values()]).sum(axis=0)
average_evaluation = evaluation_sum / len(splits)

# Save the average evaluation in the dictionary
fold_results["average"] = {
'valence_output_root_mean_squared_error': average_evaluation[0],
'arousal_output_root_mean_squared_error': average_evaluation[1]
}

# Save the fold_results dictionary to a JSON file
with open('results_across_videos.json', 'w') as outfile:
    json.dump(fold_results, outfile, indent=4)

# Print the final results
for fold_key, fold_value in fold_results.items():
    print(f"{fold_key}: {fold_value}")
